In [127]:
import torch
import torch.nn as nn
import torch.optim as optim
device=torch.device('cuda:3')

In [158]:
class LDA(nn.Module):
    def __init__(self,num_docs,num_words,num_topics,batch_size):
        super(LDA,self).__init__()
        self.num_docs=num_docs
        self.num_words=num_words
        self.num_topics=num_topics
        self.batch_size=batch_size

        self.doc_topic_prior=torch.ones(num_topics)
        self.topic_word_prior=torch.ones(num_words)
        self.doc_topic_dist=nn.Parameter(torch.randn(num_docs,num_topics))#[100,10]
        self.top_word_dist=nn.Parameter(torch.randn(num_topics,num_words))#[10,500]
    def forward(self,doc_word_counts): #[8,500]
        
        # doc_word_topic_dist=torch.zeros(self.num_docs,self.num_words,self.num_topics)
        doc_word_topic_dist = torch.zeros(self.batch_size, self.num_words, self.num_topics).to(device)
        for d in range(self.batch_size):
            prob=self.doc_topic_dist[d].unsqueeze(1) * self.top_word_dist.unsqueeze(0)
            #[1,10,500]
            # tmp=[doc_word_counts[d] for _ in range(10)]
            # tmp=torch.stack(tmp,dim=0).to(device)
            # print(prob.size())
            # print(tmp.size())#[10,500]
            # print((prob*tmp).size())
            word_count=doc_word_counts[d].unsqueeze(0).unsqueeze(0).to(device)
            doc_word_topic_dist[d]=(prob*word_count).squeeze(0).transpose(0,1)
            # print(doc_word_topic_dist[d].size())
        doc_word_topic_dist=doc_word_topic_dist/doc_word_topic_dist.sum(dim=2,keepdim=True)

        return doc_word_topic_dist


In [121]:
num_docs=100
num_words=500
num_topics=10
doc_word_counts=torch.randint(0,num_topics,(num_docs,num_words))
doc_word_counts,doc_word_counts.size()

(tensor([[1, 7, 7,  ..., 4, 0, 1],
         [2, 1, 1,  ..., 1, 7, 6],
         [1, 1, 2,  ..., 4, 8, 4],
         ...,
         [8, 4, 8,  ..., 0, 4, 9],
         [2, 8, 4,  ..., 8, 0, 8],
         [6, 3, 3,  ..., 7, 9, 9]]),
 torch.Size([100, 500]))

In [159]:
device=torch.device('cuda:3')
model=LDA(num_docs,num_words,num_topics,8).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.00000000000001)

In [117]:
doc_word_counts=doc_word_counts.to(device)
doc_word_counts

tensor([[5., 3., 7.,  ..., 6., 5., 5.],
        [2., 3., 3.,  ..., 3., 5., 2.],
        [6., 3., 5.,  ..., 4., 1., 0.],
        ...,
        [8., 2., 5.,  ..., 4., 5., 1.],
        [4., 3., 2.,  ..., 4., 5., 4.],
        [7., 8., 6.,  ..., 9., 2., 7.]], device='cuda:3')

In [25]:
doc_word_counts[0:8].size()

torch.Size([8, 500])

In [160]:
batch_size=8
num_epochs=100
num_batches = num_docs // batch_size

for epoch in range(num_epochs):
    optimizer.zero_grad()

    # 批量计算
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size
        # print(doc_word_counts[start_idx:end_idx].size())
        # 前向传播
        doc_word_topic_dist = model(doc_word_counts[start_idx:end_idx])

        # 计算损失
        loss = criterion(
            doc_word_topic_dist.view(-1, num_topics).to(device),
            doc_word_counts[start_idx:end_idx].view(-1).to(device)
        )

        # 反向传播和优化
        loss.backward()
        optimizer.step()

    # 打印训练进度
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: nan
Epoch [20/100], Loss: nan
Epoch [30/100], Loss: nan
Epoch [40/100], Loss: nan
Epoch [50/100], Loss: nan
Epoch [60/100], Loss: nan
Epoch [70/100], Loss: nan
Epoch [80/100], Loss: nan
Epoch [90/100], Loss: nan
Epoch [100/100], Loss: nan


In [135]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义LDA模型类
class LDA(nn.Module):
    def __init__(self, num_docs, num_words, num_topics, batch_size):
        super(LDA, self).__init__()
        self.num_docs = num_docs
        self.num_words = num_words
        self.num_topics = num_topics
        self.batch_size = batch_size

        # 初始化模型参数
        self.doc_topic_prior = torch.ones(num_topics)
        self.topic_word_prior = torch.ones(num_words)
        self.doc_topic_dist = nn.Parameter(torch.randn(num_docs, num_topics))
        self.topic_word_dist = nn.Parameter(torch.randn(num_topics, num_words))

    def forward(self, doc_word_counts):
        # 计算doc-word-topic概率分布
        doc_word_topic_dist = torch.zeros(self.batch_size, self.num_words, self.num_topics)
        for d in range(self.batch_size):
            doc_word_topic_dist[d] = (
                self.doc_topic_dist[d].unsqueeze(1) *
                self.topic_word_dist.unsqueeze(0) *
                doc_word_counts[d].unsqueeze(1)
            )

        # 归一化doc-word-topic概率分布
        doc_word_topic_dist = doc_word_topic_dist / doc_word_topic_dist.sum(dim=2, keepdim=True)

        return doc_word_topic_dist

# 准备数据
num_docs = 100   # 文档数量
num_words = 500  # 单词数量
num_topics = 10  # 主题数量
batch_size = 8   # 批次大小
doc_word_counts = torch.randint(0, 10, (num_docs, num_words))

# 创建LDA模型实例
model = LDA(num_docs, num_words, num_topics, batch_size)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练LDA模型
num_epochs = 100
num_batches = num_docs // batch_size

for epoch in range(num_epochs):
    optimizer.zero_grad()

    # 批量计算
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size

        # 前向传播
        doc_word_topic_dist = model(doc_word_counts[start_idx:end_idx])

        # 计算损失
        loss = criterion(
            doc_word_topic_dist.view(-1, num_topics),
            doc_word_counts[start_idx:end_idx].view(-1)
        )

        # 反向传播和优化
        loss.backward()
        optimizer.step()

    # 打印训练进度
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 打印主题-词汇分布
topic_word_dist = torch.softmax(model.topic_word_dist, dim=1)
for t in range(num_topics):
    topic_words = topic_word_dist[t].topk(5).indices
    print(f"Topic {t+1}: ", topic_words)


RuntimeError: The size of tensor a (10) must match the size of tensor b (500) at non-singleton dimension 1